In [3]:
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [4]:
model = load_model('model.keras')

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
with open('OneHotEncoder.pkl', 'rb') as file:
    OHE = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [66]:
input_data = {
    'CreditScore':600,
    'Geography':'Spain',
    'Gender': 'Male',
    'Age':40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' :2,
    'HasCrCard' : 1,
    'IsActiveMember':1,
    'EstimatedSalary' : 50000 

}

In [67]:
input_data['Geography']

'Spain'

In [68]:
OHE.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [69]:
encoded_geo = OHE.transform([[input_data['Geography']]]) ## Here 3 brackets cause we are giving it as a list
encoded_geo

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [70]:
encoded_geo_df = pd.DataFrame(encoded_geo.toarray(), columns = OHE.get_feature_names_out(['Geography']))
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [71]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,Male,40,3,60000,2,1,1,50000


In [72]:
input_data_df['Gender']= label_encoder.transform([input_data_df['Gender']])

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [73]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,1,40,3,60000,2,1,1,50000


In [74]:
input_data_df = pd.concat([input_data_df.drop("Geography", axis=1),encoded_geo_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,0.0,0.0,1.0


In [75]:
scled = scaler.transform(input_data_df)
scled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221, -0.99850112,
        -0.57946723,  1.73494238]])

In [79]:
prediction = model.predict(scled)
prob = prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [80]:
if prob > 0.5:
    print("likely to churn")
else:
    print("not likely to churn")

not likely to churn
